In [2]:
import os
import shutil
import numpy as np
import pandas as pd
from pathlib import Path

from allensdk.brain_observatory.ecephys.ecephys_project_cache import EcephysProjectCache

In [3]:
output_dir = "DATASET/"

In [4]:
# this path determines where downloaded data will be stored
manifest_path = os.path.join(output_dir, "manifest.json")

cache = EcephysProjectCache.from_warehouse(manifest=manifest_path)

print(cache.get_all_session_types())

['brain_observatory_1.1', 'functional_connectivity']


In [5]:
sessions = cache.get_session_table()

sessions.head()

,published_at,specimen_id,session_type,age_in_days,sex,full_genotype,unit_count,channel_count,probe_count,ecephys_structure_acronyms
id,,,,,,,,,,
715093703,2019-10-03T00:00:00Z,699733581,brain_observatory_1.1,118.0,M,Sst-IRES-Cre/wt;Ai32(RCL-ChR2(H134R)_EYFP)/wt,884,2219,6,"[CA1, VISrl, nan, PO, LP, LGd, CA3, DG, VISl, ..."
719161530,2019-10-03T00:00:00Z,703279284,brain_observatory_1.1,122.0,M,Sst-IRES-Cre/wt;Ai32(RCL-ChR2(H134R)_EYFP)/wt,755,2214,6,"[TH, Eth, APN, POL, LP, DG, CA1, VISpm, nan, N..."
721123822,2019-10-03T00:00:00Z,707296982,brain_observatory_1.1,125.0,M,Pvalb-IRES-Cre/wt;Ai32(RCL-ChR2(H134R)_EYFP)/wt,444,2229,6,"[MB, SCig, PPT, NOT, DG, CA1, VISam, nan, LP, ..."
732592105,2019-10-03T00:00:00Z,717038288,brain_observatory_1.1,100.0,M,wt/wt,824,1847,5,"[grey, VISpm, nan, VISp, VISl, VISal, VISrl]"
737581020,2019-10-03T00:00:00Z,718643567,brain_observatory_1.1,108.0,M,wt/wt,568,2218,6,"[grey, VISmma, nan, VISpm, VISp, VISl, VISrl]"


In [6]:
unique_areas = (
    sessions["ecephys_structure_acronyms"]
        .explode()
        .dropna()
        .unique()
)

unique_areas

array(['CA1', 'VISrl', 'PO', 'LP', 'LGd', 'CA3', 'DG', 'VISl', 'PoT',
       'VISp', 'grey', 'VISpm', 'APN', 'MB', 'VISam', 'TH', 'Eth', 'POL',
       'NOT', 'SUB', 'VL', 'CA2', 'VPM', 'VISal', 'SCig', 'PPT', 'VIS',
       'ProS', 'LGv', 'HPF', 'VISmma', 'PP', 'PIL', 'MGv', 'VPL', 'IGL',
       'SGN', 'IntG', 'LD', 'MGm', 'MGd', 'POST', 'MRN', 'VISli', 'OP',
       'ZI', 'VISmmp', 'PF', 'LT', 'RPF', 'PRE', 'SCiw', 'CP', 'COAa',
       'RT', 'SCop', 'SCsg', 'SCzo', 'COApm', 'OLF', 'BMAa'], dtype=object)

In [7]:
df_unique_areas = pd.DataFrame(unique_areas, columns=['areas'])

VIS_Area = df_unique_areas[df_unique_areas['areas'].str.startswith('VIS')]

VIS_Area

,areas
1,VISrl
7,VISl
9,VISp
11,VISpm
14,VISam
23,VISal
26,VIS
30,VISmma
43,VISli
46,VISmmp


In [8]:
VIS_Sessions = sessions[
    (sessions.ecephys_structure_acronyms.apply(lambda x: ('VISam' in x) and ('VISl' in x) and ('VISp' in x) and ('VISal' in x))) &
    (sessions.session_type.apply(lambda x: 'brain_observatory_1.1' in x))
]
print(len(VIS_Sessions))
pd_VIS_Sessions = pd.DataFrame(VIS_Sessions.ecephys_structure_acronyms)
pd_VIS_Sessions

14


,ecephys_structure_acronyms
id,
719161530,"[TH, Eth, APN, POL, LP, DG, CA1, VISpm, nan, N..."
721123822,"[MB, SCig, PPT, NOT, DG, CA1, VISam, nan, LP, ..."
750332458,"[grey, VISrl, nan, VISal, IntG, IGL, LGd, CA3,..."
750749662,"[LP, DG, CA1, VISp, nan, LGd, CA3, VISrl, VPM,..."
754312389,"[APN, POL, LP, DG, CA1, VISam, nan, LGd, HPF, ..."
755434585,"[grey, VISrl, nan, MGv, MGd, TH, LGd, CA3, DG,..."
756029989,"[TH, DG, CA3, CA1, VISl, nan, PO, Eth, LP, VIS..."
757216464,"[LP, DG, CA3, CA1, VISrl, nan, PO, LGd, HPF, V..."
760345702,"[MB, TH, PP, PIL, DG, CA3, CA1, VISal, nan, gr..."


In [9]:
session_id = 762120172
session = cache.get_session_data(session_id)

/opt/anaconda3/envs/TDA/lib/python3.11/site-packages/hdmf/spec/namespace.py:590: UserWarning: Ignoring the following cached namespace(s) because another version is already loaded:
core - cached version: 2.2.2, loaded version: 2.7.0
The loaded extension(s) may not be compatible with the cached extension(s) in the file. Please check the extension documentation and ignore this warning if these versions are compatible.
  self.warn_for_ignored_namespaces(ignored_namespaces)


In [11]:
session.units.head()

/opt/anaconda3/envs/TDA/lib/python3.11/site-packages/hdmf/spec/namespace.py:590: UserWarning: Ignoring the following cached namespace(s) because another version is already loaded:
core - cached version: 2.2.2, loaded version: 2.7.0
The loaded extension(s) may not be compatible with the cached extension(s) in the file. Please check the extension documentation and ignore this warning if these versions are compatible.
  self.warn_for_ignored_namespaces(ignored_namespaces)
/opt/anaconda3/envs/TDA/lib/python3.11/site-packages/hdmf/spec/namespace.py:590: UserWarning: Ignoring the following cached namespace(s) because another version is already loaded:
core - cached version: 2.2.2, loaded version: 2.7.0
The loaded extension(s) may not be compatible with the cached extension(s) in the file. Please check the extension documentation and ignore this warning if these versions are compatible.
  self.warn_for_ignored_namespaces(ignored_namespaces)
/opt/anaconda3/envs/TDA/lib/python3.11/site-packages

,waveform_PT_ratio,waveform_amplitude,amplitude_cutoff,cluster_id,cumulative_drift,d_prime,firing_rate,isi_violations,isolation_distance,L_ratio,...,ecephys_structure_id,ecephys_structure_acronym,anterior_posterior_ccf_coordinate,dorsal_ventral_ccf_coordinate,left_right_ccf_coordinate,probe_description,location,probe_sampling_rate,probe_lfp_sampling_rate,probe_has_lfp_data
unit_id,,,,,,,,,,,,,,,,,,,,,
951855946,0.763434,244.824840,0.000021,6,466.93,3.753336,24.461968,0.000000,155.939714,0.035349,...,8.0,grey,NaN,NaN,NaN,probeA,See electrode locations,29999.96706,1249.998627,True
951855981,0.740433,164.628555,0.004451,9,367.12,4.246110,8.642447,0.044505,83.850043,0.007322,...,8.0,grey,NaN,NaN,NaN,probeA,See electrode locations,29999.96706,1249.998627,True
951855967,0.606572,267.130110,0.006732,8,644.33,2.806621,6.212048,0.032184,40.295105,0.053282,...,8.0,grey,NaN,NaN,NaN,probeA,See electrode locations,29999.96706,1249.998627,True
951855957,0.635503,292.971315,0.001278,7,199.30,2.490130,6.375485,0.017975,39.346284,0.038742,...,8.0,grey,NaN,NaN,NaN,probeA,See electrode locations,29999.96706,1249.998627,True
951856075,0.590471,90.372360,0.017237,17,825.36,2.199514,1.210327,0.174551,35.730357,0.013001,...,8.0,grey,NaN,NaN,NaN,probeA,See electrode locations,29999.96706,1249.998627,True
